# Wigner functions

J.R. Johansson and P.D. Nation

For more information about QuTiP see [http://qutip.org](http://qutip.org)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from qutip import about, basis, destroy, mesolve, ptrace, qeye, tensor, wigner
from qutip.ipynbtools import plot_animation

%matplotlib inline

In [2]:
def jc_integrate(N, wc, wa, g, kappa, gamma, psi0, use_rwa, tlist):

    # Hamiltonian
    idc = qeye(N)
    ida = qeye(2)

    a = tensor(destroy(N), ida)
    sm = tensor(idc, destroy(2))

    if use_rwa:
        # use the rotating wave approxiation
        H = wc * a.dag() * a + wa * sm.dag() * sm + \
            g * (a.dag() * sm + a * sm.dag())
    else:
        H = wc * a.dag() * a + wa * sm.dag() * sm + \
            g * (a.dag() + a) * (sm + sm.dag())

    # collapse operators
    c_op_list = []

    n_th_a = 0.0  # zero temperature

    rate = kappa * (1 + n_th_a)
    if rate > 0.0:
        c_op_list.append(np.sqrt(rate) * a)

    rate = kappa * n_th_a
    if rate > 0.0:
        c_op_list.append(np.sqrt(rate) * a.dag())

    rate = gamma
    if rate > 0.0:
        c_op_list.append(np.sqrt(rate) * sm)

    # evolve and calculate return state vectors
    result = mesolve(H, psi0, tlist, c_op_list, [])

    return result

In [3]:
# parameters
wc = 1.0 * 2 * np.pi  # cavity frequency
wa = 1.0 * 2 * np.pi  # atom frequency
g = 0.05 * 2 * np.pi  # coupling strength
kappa = 0.05  # cavity dissipation rate
gamma = 0.15  # atom dissipation rate
N = 10  # number of cavity fock states

use_rwa = True

# start with an excited atom
psi0 = tensor(basis(N, 0), basis(2, 1))
# or a coherent state the in cavity
# psi0 = tensor(coherent(N,1.5), basis(2,0))
# or a superposition of coherent states
# psi0 = tensor((coherent(N,2.0)+coherent(N,-2.0)).unit(), basis(2,0))

tlist = np.linspace(0, 30, 150)

In [4]:
result = jc_integrate(N, wc, wa, g, kappa, gamma, psi0, use_rwa, tlist)

In [5]:
xvec = np.linspace(-5.0, 5.0, 100)
X, Y = np.meshgrid(xvec, xvec)

In [6]:
def plot_setup(result):

    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(111, projection="3d", elev=49, azim=-107)

    return fig, ax

In [7]:
cb = None


def plot_result(result, n, fig=None, axes=None):

    global cb

    if fig is None or axes is None:
        fig, ax = plot_setup(result)

    axes.cla()

    # trace out the atom
    rho_cavity = ptrace(result.states[n], 0)

    W = wigner(rho_cavity, xvec, xvec)

    surf = axes.plot_surface(
        X,
        Y,
        W,
        rstride=1,
        cstride=1,
        cmap=cm.jet,
        alpha=1.0,
        linewidth=0.05,
        vmax=0.25,
        vmin=-0.25,
    )
    axes.set_xlim3d(-5, 5)
    axes.set_ylim3d(-5, 5)
    axes.set_zlim3d(-0.25, 0.25)

    if not cb:
        cb = plt.colorbar(surf, shrink=0.65, aspect=20)

    return axes.artists

In [8]:
plot_animation(plot_setup, plot_result, result, writer="ffmpeg", codec=None)

# Versions

In [9]:
about()


QuTiP: Quantum Toolbox in Python
Copyright (c) QuTiP team 2011 and later.
Current admin team: Alexander Pitchford, Nathan Shammah, Shahnawaz Ahmed, Neill Lambert, Eric Giguère, Boxi Li, Jake Lishman, Simon Cross, Asier Galicia, Paul Menczel, and Patrick Hopf.
Board members: Daniel Burgarth, Robert Johansson, Anton F. Kockum, Franco Nori and Will Zeng.
Original developers: R. J. Johansson & P. D. Nation.
Previous lead developers: Chris Granade & A. Grimsmo.
Currently developed through wide collaboration. See https://github.com/qutip for details.

QuTiP Version:      5.1.0.dev0+c874c4a
Numpy Version:      1.22.4
Scipy Version:      1.13.1
Cython Version:     3.0.10
Matplotlib Version: 3.5.2
Python Version:     3.10.4
Number of CPUs:     4
BLAS Info:          Generic
INTEL MKL Ext:      False
Platform Info:      Linux (x86_64)
Installation path:  /usr/share/miniconda3/envs/test-environment/lib/python3.10/site-packages/qutip
Please cite QuTiP in your publication.
For your convenience a bi